# Front Matter

In [44]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import accuracy
from sklearn.metrics import precision_score, recall_score, f1_score, mean_squared_error
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split as sk_train_test_split



# 1. Data Preparation
## Load and Clean Data

In [2]:
# Load datasets
movies = pd.read_csv(r'C:\Users\pedro\Desktop\Github\DS340-Midterm\Small MovieLens\movies.csv')
ratings = pd.read_csv(r'C:\Users\pedro\Desktop\Github\DS340-Midterm\Small MovieLens\ratings.csv')
tags = pd.read_csv(r'C:\Users\pedro\Desktop\Github\DS340-Midterm\Small MovieLens\tags.csv')

In [3]:
# Display first few rows
print("Movies DataFrame:")
display(movies.head())

print("Ratings DataFrame:")
display(ratings.head())

print("Tags DataFrame:")
display(tags.head())

Movies DataFrame:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Ratings DataFrame:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Tags DataFrame:


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [4]:
# Select necessary columns
movies = movies[['movieId', 'title', 'genres']]
ratings = ratings[['userId', 'movieId', 'rating']]
tags = tags[['movieId', 'tag']]

In [5]:
# Merge Movies and Ratings to create CF DataFrame
cf = pd.merge(movies, ratings, on='movieId', how='inner')

# Merge Movies and Tags to create CBF DataFrame
cbf = pd.merge(movies, tags, on='movieId', how='inner')

In [6]:
# Check data types
print("\nData types in CF DataFrame:")
print(cf.dtypes)

print("\nData types in CBF DataFrame:")
print(cbf.dtypes)


Data types in CF DataFrame:
movieId      int64
title       object
genres      object
userId       int64
rating     float64
dtype: object

Data types in CBF DataFrame:
movieId     int64
title      object
genres     object
tag        object
dtype: object


In [7]:
# Function to extract year from title
def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        return int(match.group(1))
    else:
        return np.nan  # Handle cases where year is not found

# Apply the function to create a 'year' column
movies['year'] = movies['title'].apply(extract_year)

# Clean the 'title' by removing the year and converting to lowercase
movies['title_clean'] = movies['title'].apply(lambda x: re.sub(r'\s*\(\d{4}\)', '', x).lower())

# Verify the changes
print("Sample of Movies DataFrame after extracting year and cleaning title:")
display(movies[['movieId', 'title', 'title_clean', 'year']].head())

Sample of Movies DataFrame after extracting year and cleaning title:


,movieId,title,title_clean,year
0,1,Toy Story (1995),toy story,1995.0
1,2,Jumanji (1995),jumanji,1995.0
2,3,Grumpier Old Men (1995),grumpier old men,1995.0
3,4,Waiting to Exhale (1995),waiting to exhale,1995.0
4,5,Father of the Bride Part II (1995),father of the bride part ii,1995.0


In [8]:
# Group tags by 'movieId' and concatenate them into a single string
tags_grouped = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

# Verify the grouped tags
print("\nSample of Grouped Tags DataFrame:")
display(tags_grouped.head())


Sample of Grouped Tags DataFrame:


,movieId,tag
0,1,pixar pixar fun
1,2,fantasy magic board game Robin Williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake


In [9]:
# Merge 'tags_grouped' with 'movies' DataFrame to update 'cbf'
cbf = pd.merge(movies, tags_grouped, on='movieId', how='left')

# Replace NaN tags with empty strings (for movies without tags)
cbf['tag'] = cbf['tag'].fillna('')

# Verify the merged DataFrame
print("\nSample of Content-Based Filtering (CBF) DataFrame after merging tags:")
display(cbf.head())


Sample of Content-Based Filtering (CBF) DataFrame after merging tags:


,movieId,title,genres,year,title_clean,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,toy story,pixar pixar fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995.0,jumanji,fantasy magic board game Robin Williams game
2,3,Grumpier Old Men (1995),Comedy|Romance,1995.0,grumpier old men,moldy old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,waiting to exhale,
4,5,Father of the Bride Part II (1995),Comedy,1995.0,father of the bride part ii,pregnancy remake


In [10]:
# Create 'year_str' column for concatenation
cbf['year_str'] = cbf['year'].astype(str)

# Combine genres, title_clean, tags, and year into the 'related' column
cbf['related'] = cbf['genres'].str.replace('|', ' ') + ' ' + cbf['title_clean'] + ' ' + cbf['tag'] + ' ' + cbf['year_str']

# Verify the 'related' column
print("\nSample of CBF DataFrame with 'related' column:")
display(cbf[['movieId', 'title', 'related']].head())


Sample of CBF DataFrame with 'related' column:


,movieId,title,related
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy to...
1,2,Jumanji (1995),Adventure Children Fantasy jumanji fantasy mag...
2,3,Grumpier Old Men (1995),Comedy Romance grumpier old men moldy old 1995.0
3,4,Waiting to Exhale (1995),Comedy Drama Romance waiting to exhale 1995.0
4,5,Father of the Bride Part II (1995),Comedy father of the bride part ii pregnancy r...


In [11]:
# Preprocess the 'related' column
cbf['related'] = cbf['related'].str.lower()  # Ensure lowercase
cbf['related'] = cbf['related'].str.replace(r'\d+', '', regex=True)  # Remove numbers
cbf['related'] = cbf['related'].str.replace(r'[^a-z\s]', '', regex=True)  # Remove special characters
cbf['related'] = cbf['related'].str.strip()  # Remove extra spaces

# Verify the preprocessing
print("\nSample of CBF DataFrame after preprocessing 'related' column:")
display(cbf[['movieId', 'title', 'related']].head())


Sample of CBF DataFrame after preprocessing 'related' column:


,movieId,title,related
0,1,Toy Story (1995),adventure animation children comedy fantasy to...
1,2,Jumanji (1995),adventure children fantasy jumanji fantasy mag...
2,3,Grumpier Old Men (1995),comedy romance grumpier old men moldy old
3,4,Waiting to Exhale (1995),comedy drama romance waiting to exhale
4,5,Father of the Bride Part II (1995),comedy father of the bride part ii pregnancy r...


In [12]:
# Check for NaN values in 'related' column
nan_related = cbf['related'].isna().sum()
print(f"\nNumber of NaN values in 'related' column: {nan_related}")


Number of NaN values in 'related' column: 0


In [13]:
# Save CF and CBF DataFrames to CSV (optional)
# Uncomment if you need to save for later use
cf.to_csv(r'C:\Users\pedro\Desktop\Github\DS340-Midterm\Small MovieLens\cf.csv', index=False)
cbf.to_csv(r'C:\Users\pedro\Desktop\Github\DS340-Midterm\Small MovieLens\cbf.csv', index=False)

print("\nCleaned CF and CBF DataFrames are ready.")


Cleaned CF and CBF DataFrames are ready.


# Performing Splitting

In [48]:
# Correct usage of train_test_split with scikit-learn for a DataFrame
cbf_remaining, cbf_validation = sk_train_test_split(
    cbf, test_size=0.1, random_state=42
)

# Split the remaining data into training and testing sets (80/20 split)
cbf_train, cbf_test = sk_train_test_split(
    cbf, test_size=0.2, random_state=42
)


cf_reduced = cf[['userId', 'movieId', 'rating']]

# Define the format for the dataset using Reader
reader = Reader(rating_scale=(0.5, 5))  # Adjust the rating scale if necessary

# Load the data into Surprise format
data = Dataset.load_from_df(cf_reduced, reader)

cf_remaining, cf_validation = train_test_split(data, test_size=0.1)

# Train-test split (80% train, 20% test)
cf_train, cf_test = train_test_split(data, test_size=0.2)

# Content-Based

## Creating User Profiles

In [49]:
# Ensure 'cf_train' is a DataFrame
# If 'cf_train' is a Surprise trainset, convert it to a DataFrame
cf_train_df = cf_train.copy()

# Merge cf_train with cbf_train to get content features
user_ratings = pd.merge(cf_train_df, cbf_train[['movieId', 'related']], on='movieId', how='left')


AttributeError: 'Trainset' object has no attribute 'copy'

## Initiating TF-IDF

In [46]:
# Initialize TF-IDF Vectorizer with English stop words
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the 'related' column
tfidf_matrix = tfidf.fit_transform(cbf_train['related'])

# Verify the shape of the TF-IDF matrix
print(f"\nTF-IDF Matrix Shape: {tfidf_matrix.shape}")


TF-IDF Matrix Shape: (7793, 8624)


## Cosine Similarity

In [15]:
# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Display the similarity matrix shape (should be a square matrix: number of movies x number of movies)
print(cosine_sim.shape)

(9742, 9742)


In [16]:
# Create a function that takes in a movie title and gives recommendations
def get_recommendations(title, cosine_sim=cosine_sim):
    # Convert input title to lowercase
    title_cleaned = title.lower()

    # Find the index of the movie in the 'title_clean' column
    idx = cbf[cbf['title_clean'] == title_cleaned].index[0]

    # Get the pairwise similarity scores for all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the 10 most similar movies
    sim_scores = sim_scores[1:11]  # Skip the first movie (itself)

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the titles of the top 10 most similar movies
    return cbf['title'].iloc[movie_indices]

# Example: Get recommendations for a movie

recommendations = get_recommendations('copycat')
print(recommendations)


4805                                      Monster (2003)
1615           Henry: Portrait of a Serial Killer (1986)
296                                    Virtuosity (1995)
8672                                 Killer Movie (2008)
3128                                    Manhunter (1986)
43                           Seven (a.k.a. Se7en) (1995)
4813    Aileen: Life and Death of a Serial Killer (2003)
465                                    Serial Mom (1994)
7940                                   Killer Joe (2011)
3461                                  Others, The (2001)
Name: title, dtype: object


# Collaborative

In [17]:
# Prepare the dataset (from your CF data)
# We'll use only the necessary columns
cf_reduced = cf[['userId', 'movieId', 'rating']]

# Define the format for the dataset using Reader
reader = Reader(rating_scale=(0.5, 5))  # Adjust the rating scale if necessary

# Load the data into Surprise format
data = Dataset.load_from_df(cf_reduced, reader)

# Train-test split (80% train, 20% test)
trainset, testset = train_test_split(data, test_size=0.2)

# Initialize the SVD model for matrix factorization
svd = SVD()

# Train the model on the training set
svd.fit(trainset)

# Test the model on the test set and evaluate performance (RMSE)
predictions = svd.test(testset)
rmse = accuracy.rmse(predictions)

RMSE: 0.8680


In [18]:
# Making movie recommendations for a specific user
def recommend_movies(user_id, model=svd, n_recommendations=10):
    # Get a list of all movie IDs
    all_movie_ids = cf['movieId'].unique()

    # Predict ratings for all movies the user hasn't rated yet
    user_rated_movies = cf[cf['userId'] == user_id]['movieId'].tolist()
    unrated_movies = [movie_id for movie_id in all_movie_ids if movie_id not in user_rated_movies]

    # Predict ratings for unrated movies
    predictions = [model.predict(user_id, movie_id) for movie_id in unrated_movies]

    # Sort predictions by estimated rating in descending order
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get top N recommendations
    top_n = predictions[:n_recommendations]

    # Return movie IDs and estimated ratings
    recommended_movies = [(cf[cf['movieId'] == pred.iid]['title'].values[0], pred.est) for pred in top_n]

    return recommended_movies

# Example: Get recommendations for user 1
recommended_movies = recommend_movies(user_id=1)
print(recommended_movies)

[('Postman, The (Postino, Il) (1994)', 5), ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 5), ('Rear Window (1954)', 5), ('It Happened One Night (1934)', 5), ('North by Northwest (1959)', 5), ('Casablanca (1942)', 5), ('Bridge on the River Kwai, The (1957)', 5), ('Evil Dead II (Dead by Dawn) (1987)', 5), ('Patton (1970)', 5), ("Guess Who's Coming to Dinner (1967)", 5)]


In [19]:
cross_val_results = cross_validate(SVD(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8742  0.8723  0.8711  0.8752  0.8748  0.8735  0.0016  
MAE (testset)     0.6698  0.6704  0.6715  0.6723  0.6720  0.6712  0.0010  
Fit time          1.68    1.57    2.30    1.89    1.78    1.85    0.25    
Test time         0.12    0.20    0.33    0.17    0.26    0.22    0.07    


In [20]:
# Display the average RMSE and MAE
print("\nCross-Validation Results:")
print(f"Average RMSE: {cross_val_results['test_rmse'].mean():.4f}")
print(f"Average MAE: {cross_val_results['test_mae'].mean():.4f}")


Cross-Validation Results:
Average RMSE: 0.8735
Average MAE: 0.6712


In [21]:
# Define a parameter grid for SVD
param_grid = {
    'n_factors': [50, 100, 150],  # Number of latent factors
    'n_epochs': [20, 30],         # Number of training epochs
    'lr_all': [0.002, 0.005],     # Learning rate for all parameters
    'reg_all': [0.02, 0.05]       # Regularization term for all parameters
}

# Initialize GridSearchCV with SVD algorithm
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, joblib_verbose=1)

# Perform grid search
print("\nStarting Grid Search for hyperparameter tuning...")
gs.fit(data)
print("Grid Search completed.")

# Extract the best RMSE score
print(f"\nBest RMSE Score: {gs.best_score['rmse']:.4f}")

# Extract the best parameters
print("Best parameters:")
print(gs.best_params['rmse'])


Starting Grid Search for hyperparameter tuning...


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.2min


Grid Search completed.

Best RMSE Score: 0.8708
Best parameters:
{'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.05}


In [22]:
# Train the optimal model with best parameters
best_params = gs.best_params['rmse']
optimal_svd = SVD(
    n_factors=best_params['n_factors'],
    n_epochs=best_params['n_epochs'],
    lr_all=best_params['lr_all'],
    reg_all=best_params['reg_all']
)

print("\nTraining the optimized SVD model with best parameters...")
optimal_svd.fit(trainset)
print("Optimized model training completed.")


Training the optimized SVD model with best parameters...
Optimized model training completed.


In [23]:
# Predict on the test set using the optimized model
print("\nMaking predictions on the test set with the optimized model...")
optimal_predictions = optimal_svd.test(testset)

# Compute RMSE for the optimized model
optimal_rmse = accuracy.rmse(optimal_predictions)
print(f"Optimal Collaborative Filtering RMSE: {optimal_rmse:.4f}")


Making predictions on the test set with the optimized model...
RMSE: 0.8560
Optimal Collaborative Filtering RMSE: 0.8560


In [24]:
recommended_movies = recommend_movies(user_id=1, model = optimal_svd)
print(recommended_movies)

[('Vertigo (1958)', 5), ('His Girl Friday (1940)', 5), ('Streetcar Named Desire, A (1951)', 5), ('Third Man, The (1949)', 5), ('Ran (1985)', 5), ('On the Waterfront (1954)', 5), ('Spotlight (2015)', 5), ('Dark Knight, The (2008)', 4.997401752280771), ('Spirited Away (Sen to Chihiro no kamikakushi) (2001)', 4.992422128128404), ('Lost in Translation (2003)', 4.986724283169404)]


# Hybrid System

In [25]:
# Function to predict content-based rating for a given user and movie
def predict_cbf_rating(movie_id, cosine_sim_matrix, cbf_df, target_movie_index, user_movies, n_similar=10):
    # Get cosine similarity scores for the target movie
    sim_scores = list(enumerate(cosine_sim_matrix[target_movie_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices of the top similar movies
    sim_scores = sim_scores[1:n_similar + 1]
    
    # Calculate the weighted sum of user ratings for similar movies
    weighted_sum = 0
    sim_sum = 0
    for idx, sim_score in sim_scores:
        movie_id_similar = cbf_df.iloc[idx]['movieId']
        if movie_id_similar in user_movies:
            weighted_sum += sim_score * user_movies[movie_id_similar]
            sim_sum += sim_score
    
    # Return the weighted average rating
    if sim_sum == 0:
        return np.mean(list(user_movies.values()))  # Default to user's average rating if no similar movies
    return weighted_sum / sim_sum


In [26]:
# Hybrid recommendation: Combine CF and CBF
def hybrid_recommendation(user_id, movie_id, cf_model, cosine_sim_matrix, cbf_df, cf_df, weight_cf=0.7, weight_cbf=0.3):
    # Get the predicted CF rating (Collaborative Filtering)
    cf_prediction = cf_model.predict(user_id, movie_id).est
    
    # Get the predicted CBF rating (Content-Based Filtering)
    # First, get the movie index in CBF
    target_movie_index = cbf_df[cbf_df['movieId'] == movie_id].index[0]
    
    # Get the movies the user has rated
    user_ratings = cf_df[cf_df['userId'] == user_id].set_index('movieId')['rating'].to_dict()
    
    # Get the CBF rating prediction
    cbf_prediction = predict_cbf_rating(movie_id, cosine_sim_matrix, cbf_df, target_movie_index, user_ratings)
    
    # Combine the predictions using the weights
    final_rating = (weight_cf * cf_prediction) + (weight_cbf * cbf_prediction)
    
    return final_rating


In [27]:
def recommend_hybrid_movies(user_id, cf_model, cosine_sim_matrix, cbf_df, cf_df, n_recommendations=10, weight_cf=0.7, weight_cbf=0.3):
    all_movie_ids = cf_df['movieId'].unique()
    user_rated_movies = cf_df[cf_df['userId'] == user_id]['movieId'].tolist()
    unrated_movies = [movie_id for movie_id in all_movie_ids if movie_id not in user_rated_movies]
    
    predictions = []
    
    for movie_id in unrated_movies:
        # Predict the hybrid rating
        predicted_rating = hybrid_recommendation(user_id, movie_id, cf_model, cosine_sim_matrix, cbf_df, cf_df, weight_cf, weight_cbf)
        predictions.append((movie_id, predicted_rating))
    
    # Sort the movies by the predicted hybrid rating and get the top N
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_n_predictions = predictions[:n_recommendations]
    
    # Return the top N recommended movie titles
    return [(cbf_df[cbf_df['movieId'] == movie_id]['title'].values[0], rating) for movie_id, rating in top_n_predictions]




In [28]:
# Example: Get hybrid recommendations for user 1
recommendations = recommend_hybrid_movies(user_id=1, cf_model=optimal_svd, cosine_sim_matrix=cosine_sim, cbf_df=cbf, cf_df=cf)
recommendations

[('On the Waterfront (1954)', 5.0),
 ('Dark Knight, The (2008)', 4.9981812265965395),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
  4.962020681575945),
 ('Inglourious Basterds (2009)', 4.928702350501034),
 ("Dr. Horrible's Sing-Along Blog (2008)", 4.913298467696041),
 ('Trainspotting (1996)', 4.895338666135328),
 ('Manchurian Candidate, The (1962)', 4.890246832984927),
 ('Thank You for Smoking (2006)', 4.883977159022926),
 ('3:10 to Yuma (2007)', 4.882402227338744),
 ('Blade Runner (1982)', 4.881885194042354)]

# Validation

In [29]:
weight_cf = 0.4
weight_cbf = 1 - weight_cf

In [30]:
def generate_hybrid_predictions(testset, cf_model, cosine_sim_matrix, cbf_df, cf_df, weight_cf, weight_cbf):
    predictions = []
    for user_id, movie_id, true_rating in testset:
        # Predict the hybrid rating
        pred_rating = hybrid_recommendation(user_id, movie_id, cf_model, cosine_sim_matrix, cbf_df, cf_df, weight_cf, weight_cbf)
        predictions.append((user_id, movie_id, true_rating, pred_rating))
    return predictions
# Example: Generating predictions for a test set
hybrid_predictions = generate_hybrid_predictions(testset, optimal_svd, cosine_sim, cbf, cf, weight_cf, weight_cbf)


# RMSE and MAE

In [31]:

# Modified HybridPrediction class to mimic surprise's Prediction object
class HybridPrediction:
    def __init__(self, uid, iid, r_ui, est):
        self.uid = uid  # User ID
        self.iid = iid  # Movie ID
        self.r_ui = r_ui  # True rating
        self.est = est  # Predicted rating
        self.details = {}  # Can be empty, but required by surprise's accuracy functions

    # Making the object iterable like Surprise's Prediction class
    def __iter__(self):
        return iter((self.uid, self.iid, self.r_ui, self.est, self.details))
    
def calculate_rmse_mae(hybrid_predictions):
    # Create list of HybridPrediction objects
    surprise_predictions = [HybridPrediction(uid, iid, r_ui, est) for (uid, iid, r_ui, est) in hybrid_predictions]

    # RMSE
    rmse = accuracy.rmse(surprise_predictions, verbose=False)

    # MAE
    mae = accuracy.mae(surprise_predictions, verbose=False)

    return rmse, mae

In [32]:
rmse, mae = calculate_rmse_mae(hybrid_predictions)


# F1-Score, Precision, Recall

In [33]:
# Step 1: Prepare y_true and y_pred for binary classification
threshold = 4.5  # Set threshold for relevance

# Generate binary labels based on true ratings and predicted ratings

# Step 2: Calculate Precision, Recall, and F1-Score
def calculate_precision_recall_f1(threshold, predictions): #
    y_true = [int(true_rating >= threshold) for (_, _, true_rating, _) in predictions]  # Actual ratings
    y_pred = [int(pred_rating >= threshold) for (_, _, _, pred_rating) in predictions]  # Predicted ratings
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return precision, recall, f1
    
precision, recall, f1 = calculate_precision_recall_f1(threshold, hybrid_predictions)

In [34]:
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

RMSE: 0.8853783834734547
MAE: 0.6771916982530164
Precision: 0.6894803548795945
Recall: 0.12540341171046565
F1-Score: 0.21220986931929003
